In [1]:
import torch
# The train split of client 1
cl_num = 0
train_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/train.pt')
val_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/val.pt')
test_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/test.pt')
print(len(train_data_client) + len(val_data_client) + len(test_data_client))


1127


1: ESOL(12)
2: FreeSolv(11)
3: Lipophilicity (13)
4: BACE (7)
5: BBBP (8)
6: ClinTox (6)
7: MUTAG (1)
8: PTC_MR (3)
9: PTC_MM (2)
10:  PTC_FM (4)
11: PTC_FR (5) Zu viele Datem: 1440
12: NCI109 (10)
13: NCI1 (9)
14: alchemy_full (15)
15: ZINC_full (16)
16: QM9 (14) (zu viele Daten? 130831)

In [1]:

from federatedscope.register import register_trainer
from federatedscope.contrib.trainer.FedAvg_VAE_trainer import call_fedavg_VAE_trainer

register_trainer('FedAvg_VAE_trainer', call_fedavg_VAE_trainer)

In [2]:
import os
import sys

# from federatedscope.core.cmd_args import parse_args
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.auxiliaries.worker_builder import get_client_cls, get_server_cls
from federatedscope.core.configs.config import global_cfg
from federatedscope.core.fed_runner import FedRunner
from yacs.config import CfgNode
from rdkit.Chem.Draw import IPythonConsole
from IPython.display import clear_output


if os.environ.get('https_proxy'):
    del os.environ['https_proxy']
if os.environ.get('http_proxy'):
    del os.environ['http_proxy']

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
2023-01-02 20:06:41,086 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [3]:
def train(client, lr):
    cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/fedavg_VAE.yaml'
    cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_isolated.yaml'
    #'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

    init_cfg = global_cfg.clone()
    init_cfg.merge_from_file(cfg_file)

    # init_cfg.data.subdirectory = 'graph_dt_backup/processed'
    # init_cfg.merge_from_list(args.opts)
    init_cfg.data.client = client
    init_cfg.train.optimizer.lr = lr
    init_cfg.data.save_dir = 'test'
    update_logger(init_cfg)
    setup_seed(init_cfg.seed)

    # federated dataset might change the number of clients
    # thus, we allow the creation procedure of dataset to modify the global cfg object
    data, modified_cfg = get_data(config=init_cfg.clone())
    init_cfg.merge_from_other_cfg(modified_cfg)

    init_cfg.freeze()

    # allow different settings for different clients
    # cfg_client.merge_from_file(args.cfg_client)
    if cfg_client is None:
        cfg_client = None
    else:
        cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()
    runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
    _ = runner.run()

In [4]:
clients = range(1, 1+1)
lrs = [0.01]
num_trainings = 3
for client in clients:
    clear_output(wait=True)
    for lr in lrs:
        print(f"Client: {client},\tlr: {lr}")
        for i in range(num_trainings):
            print(f"training run: {i+1}")
            train(client, lr)

2023-01-02 19:40:04,212 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2023-01-02 19:40:04,213 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-02 19:40:04,215 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-02 19:40:04,215 (utils:132)INFO: the output dir is exp/test/local_gin_on_graph-dt_lr0.01_lstep1_/sub_exp_20230102194004


Client: 1,	lr: 0.01
training run: 1


2023-01-02 19:40:04,854 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2023-01-02 19:40:04,855 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2023-01-02 19:40:04,864 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  client: 1
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    

TypeError: Unsupported data type.

In [6]:
from federatedscope.core.configs import CN

cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/fedavg_VAE.yaml'
cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml'
#'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

init_cfg = global_cfg.clone()
init_cfg.merge_from_file(cfg_file)
init_cfg.federate.client_num = 1
init_cfg.params = CN()
init_cfg.params.alpha=0.1
# init_cfg.data.subdirectory = 'graph_dt_backup/processed'
# init_cfg.merge_from_list(args.opts)
#init_cfg.data.client = 5
#init_cfg.train.optgraph_level_defaultimizer.lr = 0.01
init_cfg.data.save_dir = 'test_dir'
update_logger(init_cfg)
setup_seed(init_cfg.seed)

# federated dataset might change the number of clients
# thus, we allow the creation procedure of dataset to modify the global cfg object
data, modified_cfg = get_data(config=init_cfg.clone())
init_cfg.merge_from_other_cfg(modified_cfg)

init_cfg.freeze()

# allow different settings for different clients
# cfg_client.merge_from_file(args.cfg_client)
if cfg_client is None:
    cfg_client = None
else:
    cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()



2023-01-02 20:08:09,938 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-02 20:08:09,939 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-02 20:08:09,940 (utils:132)INFO: the output dir is exp/test_dir/Graph-DT-FedAvg_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230102200809
2023-01-02 20:08:10,869 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:

In [7]:
1


1

In [9]:
from federatedscope.contrib.workers.fedavg_VAE_client import Fedavg_VAE_client

runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=Fedavg_VAE_client,
                   config=init_cfg.clone(),
                   client_config=cfg_client,)
_ = runner.run()

2023-01-02 20:09:07,361 (worker_builder:73)WARNING: Server for method Graph-DT-FedAvg is not implemented. Will use default one
2023-01-02 20:09:07,526 (aggregator_builder:21)WARNING: Aggregator for method Graph-DT-FedAvg is not implemented. Will use default one
2023-01-02 20:09:07,527 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-02 20:09:07,582 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 1
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  qua

vae_loss: 2.3140084743499756
vae_loss: 2.302180290222168
vae_loss: 2.2623884677886963
vae_loss: 2.248159408569336
vae_loss: 2.202038049697876
vae_loss: 2.1752772331237793
vae_loss: 2.1553525924682617
vae_loss: 2.1193735599517822
vae_loss: 2.0919835567474365
vae_loss: 2.0689194202423096
vae_loss: 2.047959804534912
vae_loss: 2.0223851203918457
vae_loss: 1.9829695224761963
vae_loss: 1.9727274179458618
vae_loss: 1.945608377456665


2023-01-02 20:10:06,095 (client:255)INFO: {'Role': 'Client #1', 'Round': 0, 'Results_raw': {'train_avg_loss': 12.029993, 'train_total': 901, 'train_imp_ratio': -1201.524728, 'train_loss': 10839.02357}}
2023-01-02 20:10:06,106 (server:323)INFO: Server #0: Starting evaluation at the end of round 0.
2023-01-02 20:10:06,114 (server:330)INFO: ----------- Starting a new training round (Round #1) -------------
2023-01-02 20:10:06,211 (client:410)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'test_avg_loss': 9.239628, 'test_total': 113, 'test_imp_ratio': -899.635269, 'test_loss': 1044.077963, 'val_avg_loss': 9.242317, 'val_total': 113, 'val_imp_ratio': -899.926231, 'val_loss': 1044.38183}}
2023-01-02 20:10:06,213 (monitor:513)INFO: current_best=-899.926231, should_save=True
2023-01-02 20:10:06,214 (client:431)INFO: Client: #1, val_imp_ratio: -899.926231. model saved at exp/test_dir/Graph-DT-FedAvg_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230102200809/model1.pth


vae_loss: 1.6548593044281006
vae_loss: 1.6402397155761719
vae_loss: 1.6416888236999512
vae_loss: 1.6457631587982178
vae_loss: 1.9231219291687012
vae_loss: 1.9164633750915527
vae_loss: 1.8654006719589233
vae_loss: 1.8638417720794678
vae_loss: 1.8239482641220093
vae_loss: 1.7951691150665283
vae_loss: 1.7942075729370117


2023-01-02 20:10:06,519 (client:255)INFO: {'Role': 'Client #1', 'Round': 1, 'Results_raw': {'train_avg_loss': 8.032604, 'train_total': 901, 'train_imp_ratio': -769.047303, 'train_loss': 7237.376154}}
2023-01-02 20:10:06,525 (server:496)INFO: {'Role': 'Server #', 'Round': 1, 'Results_avg': {'test_avg_loss': 9.239628, 'test_total': 113.0, 'test_imp_ratio': -899.635269, 'test_loss': 1044.077963, 'val_avg_loss': 9.242317, 'val_total': 113.0, 'val_imp_ratio': -899.926231, 'val_loss': 1044.38183}}
2023-01-02 20:10:06,526 (monitor:513)INFO: current_best=-10000, should_save=False
2023-01-02 20:10:06,528 (monitor:513)INFO: current_best=-899.926231, should_save=True
2023-01-02 20:10:06,536 (server:323)INFO: Server #0: Starting evaluation at the end of round 1.
2023-01-02 20:10:06,545 (server:330)INFO: ----------- Starting a new training round (Round #2) -------------


...

2023-01-02 20:15:51,862 (client:410)INFO: {'Role': 'Client #1', 'Round': 241, 'Results_raw': {'test_avg_loss': 3.027561, 'test_total': 113, 'test_imp_ratio': -227.551738, 'test_loss': 342.114371, 'val_avg_loss': 2.601714, 'val_total': 113, 'val_imp_ratio': -181.479353, 'val_loss': 293.993665}}
2023-01-02 20:15:51,863 (monitor:513)INFO: current_best=-158.901245, should_save=False


vae_loss: 0.22489678859710693
vae_loss: 0.22483029961585999
vae_loss: 0.2019309252500534
vae_loss: 0.20812638103961945
vae_loss: 0.2086789309978485
vae_loss: 0.1996435523033142
vae_loss: 0.24681764841079712
vae_loss: 0.20387208461761475
vae_loss: 0.19987741112709045
vae_loss: 0.22224494814872742
vae_loss: 0.21675202250480652
vae_loss: 0.21935845911502838


2023-01-02 20:15:52,159 (client:255)INFO: {'Role': 'Client #1', 'Round': 241, 'Results_raw': {'train_avg_loss': 3.374652, 'train_total': 901, 'train_imp_ratio': -265.103554, 'train_loss': 3040.561648}}
2023-01-02 20:15:52,163 (server:496)INFO: {'Role': 'Server #', 'Round': 241, 'Results_avg': {'test_avg_loss': 3.027561, 'test_total': 113.0, 'test_imp_ratio': -227.551738, 'test_loss': 342.114371, 'val_avg_loss': 2.601714, 'val_total': 113.0, 'val_imp_ratio': -181.479353, 'val_loss': 293.993665}}
2023-01-02 20:15:52,164 (monitor:513)INFO: current_best=-158.901245, should_save=False
2023-01-02 20:15:52,165 (monitor:513)INFO: current_best=-158.901245, should_save=False
2023-01-02 20:15:52,171 (server:323)INFO: Server #0: Starting evaluation at the end of round 241.
2023-01-02 20:15:52,178 (server:330)INFO: ----------- Starting a new training round (Round #242) -------------


vae_loss: 0.20224495232105255
vae_loss: 0.2046225666999817
vae_loss: 0.2476302981376648
vae_loss: 0.22130127251148224


KeyboardInterrupt: 

In [8]:
data

{1: {'train': <torch_geometric.loader.dataloader.DataLoader at 0x7fa4a64be790>,
  'val': <torch_geometric.loader.dataloader.DataLoader at 0x7fa4a66cbca0>,
  'test': <torch_geometric.loader.dataloader.DataLoader at 0x7fa5b9984af0>,
  'num_label': 0}}

In [ ]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)


In [ ]:
cfg_client['client_5']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_6']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_7']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_8']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_9']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_10']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_11']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_12']['eval']['metrics']=['mse']

In [ ]:
cfg_client['client_13']['eval']['metrics']=['mse']

In [ ]:
cl_list = ['client_' + str(i) for i in range(1,17)]

In [ ]:
cl_list

In [ ]:
cfg_client['client_13']['train']['local_update_steps']

In [ ]:
for el in cl_list:
    cfg_client[el]['train']['local_update_steps']=1

In [ ]:
cfg_client